In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from utils import *
from imblearn.under_sampling import RandomUnderSampler
from fancyimpute import KNN

import warnings
warnings.filterwarnings('ignore')

In [8]:
%%time
##### load & clean data
train, test = load_data()
train = train.dropna(axis=0, how='all') # remove missing rows
y_all = train.Class
train = train.drop(columns='Class')

#columns_to_drop = ['Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
#train, test = remove_columns(train, test, columns_to_drop)

print("train.shape without missing rows =", train.shape)
print("class count:\n", y_all.value_counts())
##### add here new steps before predict (TODO: undersample)
##### add here new steps to preprocess
preprocessing = Pipeline([
                ('scaler', StandardScaler()),
                ('knn-imputation', KNN(k=5, print_interval=10000))
                ])
X_all, X_test = preprocess_data(train, test, preprocessing)
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size = 0.2, stratify=y_all, random_state=42)

xg = XGBClassifier(n_estimators=40, 
                   tree_method='gpu_exact', 
                   predictor='gpu_predictor', 
                   random_state=52, 
                   scale_pos_weight=1.2,
                   max_depth=4, 
                   n_jobs=-1)

evaluation =  Pipeline([
#                 ('smote', SMOTE(random_state=42, ratio=1, n_jobs=-1, k_neighbors=3)),
#                 ('random_oversample', RandomOverSampler()),
                    ('xg', xg)
               # ('select_from_model',SelectFromModel(selection_model, threshold=0.071, prefit=True))
                ])
# evaluation.get_params().keys()
gs = evaluation
gs.fit(X_train, y_train)  
# print('best params:\n\t{}'.format(gs.best_params_))
# print('best auc score:\n\t{:.4f}'.format(gs.best_score_))

x_val_pred = gs.predict_proba(X_val)[:,1]
print('auc score on x_val:\n\t{:.4f}'.format(roc_auc_score(y_val, x_val_pred)))
gs.fit(X_all, y_all)
pred = gs.predict_proba(X_test)[:,1]
# to_file(pred, "tal_test_k5")

train.shape without missing rows = (30451, 51)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64
Imputing row 1/42903 with 10 missing, elapsed time: 827.372
Imputing row 10001/42903 with 10 missing, elapsed time: 851.094
Imputing row 20001/42903 with 11 missing, elapsed time: 869.525
Imputing row 30001/42903 with 14 missing, elapsed time: 887.584
Imputing row 40001/42903 with 18 missing, elapsed time: 905.441
auc score on x_val:
	0.6736
Wall time: 15min 15s
